## CIFAR10 데이터셋을 가지고 연습해봅시다

In [2]:
%matplotlib inline

# A. logistic regression

## 0. from scratch

### data loading 

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [5]:
from matplotlib import pyplot as plt
import numpy as np

In [6]:
img = trainset.data[1201]
plt.imshow(img)

In [7]:
trainset.targets[17865]

0

### data preprocessing 

In [8]:
x_train, y_train, x_test, y_test  = map(torch.Tensor, (trainset.data, trainset.targets,
                                     testset.data, testset.targets))

In [9]:
# indexing의 자료형은 int형이여야 함. (when doing code from scratch)
# y_label의 nll loss를 구할 때 category가 int형이여야 함. (when using functional)
y_train = y_train.type(torch.int64)
y_test = y_test.type(torch.int64)

In [10]:
# n : number of data set
# h : height of image
# w : weight of image
# c : channel of image
n, h, w, c = x_train.shape
n_categories = int(y_train.max() - y_train.min() + 1)

In [11]:
x_train.shape

torch.Size([50000, 32, 32, 3])

In [12]:
import math 

nodes = h * w * c
weights = torch.randn(nodes, n_categories) / math.sqrt(nodes)

In [13]:
weights

tensor([[-0.0290, -0.0001,  0.0098,  ..., -0.0135, -0.0066, -0.0076],
        [-0.0178,  0.0219,  0.0081,  ..., -0.0060, -0.0057,  0.0149],
        [-0.0145,  0.0069,  0.0160,  ..., -0.0272,  0.0128,  0.0326],
        ...,
        [-0.0215,  0.0019,  0.0050,  ..., -0.0031, -0.0101,  0.0208],
        [ 0.0224,  0.0041, -0.0009,  ..., -0.0190, -0.0289, -0.0248],
        [ 0.0079,  0.0188,  0.0101,  ...,  0.0542, -0.0120, -0.0004]])

In [14]:
print(weights.requires_grad)
weights.requires_grad_()
print(weights.requires_grad)

False
True


In [15]:
bias = torch.zeros(n_categories, requires_grad = True)

$$ LogSoftmax(x)_i = log(\frac{e^{x_i}}{\sum_{j=1}^{K}{e^{x_j}}}) $$

In [16]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1) # broadcasting
# berfore unsqueeze shape : (n, )
# after unsqueeze shape   : (n, 1) 

In [17]:
weights.exp().sum(-1).log().unsqueeze(-1).shape

torch.Size([3072, 1])

#### mnist data set과 달리 3차원이기 때문에 1차원으로 바꿔줌 : torch.flatten 함수

In [18]:
def preprocess(xb):
    return torch.flatten(xb, start_dim=1)

In [19]:
def model(xb):
    xb = preprocess(xb)
    return log_softmax(xb @ weights + bias)

In [20]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()

loss_func = nll

In [21]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [22]:
lr = 0.5
epochs = 2

In [23]:
bs = 1000

In [24]:
for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

In [25]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(nan, grad_fn=<NegBackward>) tensor(0.0870)


## 1. torch.nn.functional 사용하기
loss function을 functional에서 가져와서 써줌. 

In [26]:
import torch.nn.functional as F

In [27]:
loss_func = F.cross_entropy

In [28]:
def model(xb):
    xb = preprocess(xb)
    return xb @ weights + bias

In [29]:
for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        yb = yb.type(torch.int64) # indexing의 자료형은 int64여야함.
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

In [30]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(nan, grad_fn=<NllLossBackward>) tensor(0.0870)


## 2. nn.Module을 사용해보기 
nn.Module을 상속하여 클래스를 만든다<br>
forward 단계에서 가중치, 절편, method를 유지하고, ``.parameters()``와 ``.zero_grad()`` 등을 가지고 있음<br>

In [31]:
from torch import nn

In [32]:
class CIFAR_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(nodes, n_categories) / math.sqrt(nodes))
        self.bias = nn.Parameter(torch.zeros(n_categories))
    
    def forward(self, xb):
        xb = preprocess(xb)
        return xb @ self.weights + self.bias

함수를 사용하는 대신에, 오브젝트를 사용함<br>
``nn.Module``오브젝트들은 마치 함수처럼 사용할 수 있다. 배후에서 PyTorch는 우리의 ``forward``함수를 자동으로 호출함

In [33]:
model = CIFAR_Logistic()

In [34]:
def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

                
fit()

In [35]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(4124707.5000, grad_fn=<NllLossBackward>) tensor(0.1740)


## 3. nn.Linear를 사용해보기
weight, bias를 정의 및 초기화하고, ``xb @ weights + bias``를 계산하는 대신에 미리 정의된 선형 레이어로 ``nn.Linear``를 사용함

In [36]:
class CIFAR_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(nodes, n_categories)
        
    def forward(self, xb):
        return self.linear(preprocess(xb))

In [37]:
model = CIFAR_Logistic()
fit()
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(5978771.5000, grad_fn=<NllLossBackward>) tensor(0.1810)


## 4. optim을 사용해보기 
매개변수를 수동으로 업데이트 하는 대신 옵티마이저의 ``step``메서드를 통해 업데이트 가능<br>
``optim.zero_grad()``는 기울기를 0으로 재설정해주어 다음 미니 배치의 기울기 계산 전에 호출해야 함

In [38]:
from torch import optim

In [39]:
def get_model():
    model = CIFAR_Logistic()
    return model, optim.SGD(model.parameters(), lr=lr)

In [40]:
model, opt = get_model()
print(loss_func(model(xb), yb))

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

tensor(76.9773, grad_fn=<NllLossBackward>)


## 5. Dataset을 사용해보기
x와 y를 묶어서 코드 한 줄에서 사용하기 용이<br>
이 전에는 x의 배치와 y와 배치를 따로 작성했다면 이번엔 한번에 인덱싱할 수 있음

In [41]:
from torch.utils.data import TensorDataset

In [42]:
train_ds = TensorDataset(x_train, y_train)

In [43]:
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        xb, yb = train_ds[i * bs: i * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(4439417.5000, grad_fn=<NllLossBackward>)


## 6. DataLoader를 사용해보기
``DataLoader``는 배치 관리를 담당하여 반복하기 쉽게 만들어 줌

In [44]:
from torch.utils.data import DataLoader

In [45]:
train_dl = DataLoader(train_ds, batch_size=bs)

In [46]:
model, opt = get_model()
for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(4667869., grad_fn=<NllLossBackward>)


## 7. test data추가하기

In [47]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_test, y_test)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

In [48]:
model, opt = get_model()

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

0 tensor(4965318.)
1 tensor(4535693.5000)


## 8. refactoring : fit(), get_data()

In [49]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [50]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

In [51]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

In [52]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 6814374.0
1 3717927.9


# B. CNN
사전에 정의된 ``nn.Conv2d``를 사용함<br>
Conv2D + ReLU + average pooling을 진행함

## 1. nn.Module + nn.Conv2d 

In [53]:
class CIFAR_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # nn.Conv2d(in_channels, out_channels)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1) # RGB channel 개수가 1->3 이어서 변경해줌
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, n_categories, kernel_size=3, stride=2, padding=1)
    
    def forward(self, xb):
        xb = xb.view(-1, c, w, h) # [1000, 32, 32, 3] -> [1000, 3, 32, 32]
        xb = F.relu(self.conv1(xb)) # [1000, 3, 32, 32] -> [1000, 16, 16, 16]
        xb = F.relu(self.conv2(xb)) # [1000, 16, 16, 16] -> [1000, 16, 8, 8]
        xb = F.relu(self.conv3(xb)) # [1000, 16, 8, 8] -> [1000, 10, 4, 4] 
        xb = F.avg_pool2d(xb, 4) # [1000, 10, 4, 4] -> [1000, 10, 1, 1]
        return xb.view(-1, xb.size(1))

In [54]:
model = CIFAR_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 2.302568197250366
1 2.302568197250366


## 2. nn.Sequential 사용해보기
``Sequential``을 사용하면 해당 객체에 포함된 각 모듈을 순차적으로 실행함<br>
이를 활용하려면 사용자정의 레이어를 정의해야 함

In [55]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def preprocess(x):
    return x.view(-1, c, w, h)

In [56]:
model = nn.Sequential(
    Lambda(preprocess),
    nn.Conv2d(c, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, n_categories, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(4), # 입력 텐서 크기 
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 2.302568197250366
1 2.302568197250366


## 3. DataLoader 감싸기
preprocess를 모델에서 dataloader로 옮길 수 있습니다.

In [57]:
def preprocess(x, y):
    return x.view(-1, c, w, h), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

In [58]:
model = nn.Sequential(
    nn.Conv2d(c, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1), # AdaptiveAvgPool2d는 출력텐서의 크기를 정의할 수 있음
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [59]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 2.302568197250366
1 2.302568197250366
